In [ ]:
import pandas as pd
import numpy as np
import gspread
from google.oauth2.service_account import Credentials
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

# Google Sheets API Setup
SERVICE_ACCOUNT_FILE = "/content/leaf-corp-d0623cb78464.json"  # Replace with your service account JSON file
SCOPES = ["https://www.googleapis.com/auth/spreadsheets"]

creds = Credentials.from_service_account_file(SERVICE_ACCOUNT_FILE, scopes=SCOPES)
client = gspread.authorize(creds)

# Open the Google Sheet by ID
SHEET_ID = "14g98RZnbqtrIfxyihWw6AWhOGLwcqh_1MOz-aryR6lA"  # Replace with actual Sheet ID
SHEET_NAME = "Chilli_data_train"  # Change if necessary
sheet = client.open_by_key(SHEET_ID).worksheet(SHEET_NAME)

# Load data into DataFrame
data = sheet.get_all_values()
df = pd.DataFrame(data[1:], columns=data[0])  # First row as column names

# Convert numeric columns
df.iloc[:, 1:] = df.iloc[:, 1:].apply(pd.to_numeric, errors='coerce')

# Function to categorize values into Low, Normal, and High
def categorize_features(data):
    categorized_data = data.copy()
    thresholds = {}  # Store threshold values

    for col in data.columns:
        low_thresh = np.percentile(data[col], 33)  # 33rd percentile
        high_thresh = np.percentile(data[col], 67)  # 67th percentile
        thresholds[col] = (low_thresh, high_thresh)

        categorized_data[col] = np.select(
            [data[col] < low_thresh, data[col] >= high_thresh],
            ['Low', 'High'],
            default='Normal'
        )

    return categorized_data, thresholds

# Extract only relevant feature columns
def process_common_data(df):
    feature_columns = ['temperature', 'humidity', 'soil moisture', 'pH']
    df = df[feature_columns]  # Keep only necessary columns
    df = df.apply(pd.to_numeric, errors='coerce')  # Convert to numeric
    df = df.dropna()  # Drop missing values
    return df

# Process training data
df = process_common_data(df)

# Convert numerical values to categories
categorized_df, thresholds = categorize_features(df)

# Prepare Features (X) and Target (y)
X = df  # Numerical dataset
y = categorized_df  # Categorical dataset (Low/Normal/High)

# Split data (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train a Decision Tree Classifier for each feature
models = {}
for column in y.columns:
    model = DecisionTreeClassifier(max_depth=5, random_state=42)
    model.fit(X_train, y_train[column])
    models[column] = model

# Function to predict categories on new test data
def predict_categories(test_sheet_id, test_sheet_name):
    test_sheet = client.open_by_key(test_sheet_id).worksheet(test_sheet_name)
    test_data = test_sheet.get_all_values()
    test_df = pd.DataFrame(test_data[1:], columns=test_data[0])

    # Process only common columns
    test_df = process_common_data(test_df)

    # Make predictions for each feature
    predictions = {}
    for feature, model in models.items():
        predictions[feature] = model.predict(test_df)

    results = pd.DataFrame(predictions)
    return results

# Test the model with a new dataset from Google Sheets
TEST_SHEET_ID = "1XdXlgTWxEDfU0uHK1Z3mM885JQuM0EH16DrDaIHP3zc"  # Replace with test Google Sheet ID
TEST_SHEET_NAME = "Sheet1"

predictions = predict_categories(TEST_SHEET_ID, TEST_SHEET_NAME)

print("\n🔹 Predictions from Test Dataset:")
print(predictions)

# Evaluate model accuracy
accuracy = {}
for column in y.columns:
    y_pred = models[column].predict(X_test)
    accuracy[column] = accuracy_score(y_test[column], y_pred)

print("\n🔹 Model Accuracy per Feature:")
for feature, acc in accuracy.items():
    print(f"{feature}: {acc:.2f}")

# Show threshold values used
print("\n🔹 Thresholds Used for Categorization:")
for feature, (low, high) in thresholds.items():
    print(f"{feature}: Low < {low:.2f}, Normal ({low:.2f} - {high:.2f}), High > {high:.2f}")



🔹 Predictions from Test Dataset:
   temperature humidity soil moisture      pH
0         High      Low          High  Normal
1         High      Low          High  Normal
2         High      Low          High  Normal
3         High      Low          High  Normal
4         High      Low          High  Normal
..         ...      ...           ...     ...
68        High      Low          High  Normal
69        High      Low          High  Normal
70        High      Low          High  Normal
71        High      Low          High  Normal
72        High      Low          High  Normal

[73 rows x 4 columns]

🔹 Model Accuracy per Feature:
temperature: 0.95
humidity: 1.00
soil moisture: 0.95
pH: 0.95

🔹 Thresholds Used for Categorization:
temperature: Low < 22.69, Normal (22.69 - 24.89), High > 24.89
humidity: Low < 81.44, Normal (81.44 - 83.04), High > 83.04
soil moisture: Low < 218.41, Normal (218.41 - 257.01), High > 257.01
pH: Low < 6.01, Normal (6.01 - 6.94), High > 6.94
